In [ ]:
from Ichigaya import chart
刹那旅行 = chart.Chart(77, "hard")
刹那旅行.to_path("谱面")
json = 刹那旅行.load()

In [ ]:
刹那旅行.b_s_trans(285.1)

In [ ]:
chart_keys = {
    "Single": [], 
    "Hold": [], 
    "Flick": [], 
    "Hold&Flick": [], 
    "Left": [], 
    "Right": []
}
time_stamps = []
for i in json:
    if i["type"] == "BPM":
        time_stamps.append(i)
    elif i["type"] == "Single":
        if "flick" in i.keys():
            chart_keys["Flick"].append(i)
        else:
            chart_keys["Single"].append(i)
    elif i["type"] == "Directional":
        if i["direction"] == "Left":
            chart_keys["Left"].append(i)
        else:
            chart_keys["Right"].append(i)
    elif i["type"] != "System":
        if "flick" in i["connections"][-1].keys():
            chart_keys["Hold&Flick"].append(i)
        else:
            chart_keys["Hold"].append(i)


In [ ]:
for i in 刹那旅行.schedule:
    print(i[2] if type(i) == tuple else i.beat)

In [ ]:
x = 刹那旅行.get_min_retouch()

In [ ]:
print(x)

In [ ]:
刹那旅行.get_len()

In [ ]:
刹那旅行.b_s_trans(221.5729)

In [ ]:
shadow = {}
for single in chart_keys["Single"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

In [ ]:
for hold in chart_keys["Hold"]:
    for node in [hold["connections"][0], hold["connections"][-1]]:
        beat = str(node["beat"])
        if beat in shadow.keys():
            shadow[beat].append(node["lane"])
        else:
            shadow.update({beat: [node["lane"]]})

In [ ]:
for hold in chart_keys["Hold&Flick"]:
    for node in [hold["connections"][0], hold["connections"][-1]]:
        beat = str(node["beat"])
        if beat in shadow.keys():
            shadow[beat].append(node["lane"])
        else:
            shadow.update({beat: [node["lane"]]})

In [ ]:
for single in chart_keys["Flick"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

for single in chart_keys["Left"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

for single in chart_keys["Right"]:
    beat = str(single["beat"])
    if beat in shadow.keys():
        shadow[beat].append(single["lane"])
    else:
        shadow.update({beat: [single["lane"]]})

In [ ]:
links = {}
for stamp in shadow.keys():
    if len(shadow[stamp]) > 1:
        links.update({stamp: shadow[stamp]})
print(links)

In [ ]:
key_shape = {
    "Single": " (===========) ", 
    "Flick": " (^^^^^^^^^^^) ", 
    "Hold_start": " /===========\\ ", 
    "Hold_end": " \\===========/ ", 
    "Hold_body": "  +         +  ", 
    "Hold_node": "  +   ---   +  ", 
    "Left": [
        " <===========0 ", 
        " <<<=======================0   ", 
        " <<<<<===================================0     "
    ], 
    "Right": [
        " 0===========> ", 
        "   0=======================>>> ", 
        "     0===================================>>>>> "
    ]
}
chart_pos = {
    "left": [1, 17, 33, 49, 65, 81, 97], 
    "right": [15, 31, 47, 63, 79, 95, 111],
    "middel": [8, 24, 40, 56, 72, 88, 104] 
}

In [ ]:
bpm = 145
length = 288 + 1
freq = 60 / bpm
expand = 1 / freq
if expand < 1.41: bps = 1
elif expand < 2.82: bps = 2
else: bps = 4
lpb = int(64 / bps)
chart_line = lpb * length
blank = ("|" + " " * 15)*7 + "|"
chart_text = [blank] * chart_line

In [ ]:
for single in chart_keys["Single"]:
    line = int((float(single["beat"]) * lpb))
    lane = single["lane"]
    node = chart_pos["left"][lane], chart_pos["right"][lane]
    line_chart = chart_text[line]
    line_chart = line_chart[:node[0]] + key_shape["Single"] + line_chart[node[1]+1:]
    chart_text[line] = line_chart

In [ ]:
for single in chart_keys["Flick"]:
    line = int((float(single["beat"]) * lpb))
    lane = single["lane"]
    node = chart_pos["left"][lane], chart_pos["right"][lane]
    line_chart = chart_text[line]
    line_chart = line_chart[:node[0]] + key_shape["Flick"] + line_chart[node[1]+1:]
    chart_text[line] = line_chart

In [ ]:
for beat in links.keys():
    line = int((float(beat) * lpb))
    if links[beat][0] < links[beat][1]:
        node = chart_pos["middel"][links[beat][0]], chart_pos["middel"][links[beat][1]]
    else: 
        node = chart_pos["middel"][links[beat][1]], chart_pos["middel"][links[beat][0]]
    line_chart = chart_text[line]
    new_line_chart = ""
    for i in range(len(line_chart)):
        if line_chart[i] in [" ", "|"] and (i >= node[0] and i <= node[1]):
            new_line_chart += "-"
        else:
            new_line_chart += line_chart[i]
    chart_text[line] = new_line_chart

In [ ]:
time_step = [0] * len(chart_text)
for stamp in time_stamps:
    index = int((float(stamp["beat"]) * lpb))
    step = 60 / float(stamp["bpm"]) / lpb
    for i in range(index, len(chart_text)):
        time_step[i] = step

In [ ]:
line_view = 64
line_stride = int(line_view / 8)
time_stride = []
display_charts = []
for i in range(0, len(chart_text) - line_view, line_stride):
    time_stride.append(
        sum(time_step[i:i + line_stride]))
    display_charts.append(
        [chart_text[i: i + line_view]]
    )
    display_charts[-1] = display_charts[-1][::-1]

In [ ]:
a = [1,2,3,6,5,4]

In [ ]:
from Ichigaya.chart import get_charts
all_charts = get_charts(diff = ["expert", "special"], process = True)

In [ ]:
all_charts = sorted(all_charts, key = lambda chart: chart.max_combo, reverse = True)
for chart in all_charts[:20]:
    print(chart.get()[0], chart.get()[1], chart.max_combo)

In [ ]:
all_charts = sorted(all_charts, key = lambda chart: chart.amount, reverse = True)
for chart in all_charts[:20]:
    print(chart.get()[0], chart.get()[1], chart.amount)

In [ ]:
from Ichigaya import id_name_trans

In [ ]:
id_name_trans("LOUDER")

In [ ]:
刹那旅行.get_point_list()

In [ ]:
x = [1, 2, 3, 4, 5]
x.pop(3)
x

In [ ]:
for simo in 刹那旅行.simo:
    print(simo.obj1, simo.obj2, simo.beat, simo.lane)

In [ ]:
for i, point in enumerate(刹那旅行.get_points()):
    if point[2] == 0:
        print(i, point)

In [ ]:
for i in 刹那旅行.keys["Flick"]:
    print(i.beat, i.lane)

In [ ]:
刹那旅行.schedule

In [ ]:
pl = 刹那旅行.get_point_list()
pl = sorted(pl, key = lambda point: point[2])
pl

In [ ]:
s = 刹那旅行.keys["Single"]
s = sorted(s, key = lambda sin: sin.beat)

In [ ]:
from Ichigaya import chart
for i in range(500):
    test_chart = chart.Chart(i + 1, "special")
    test_chart.to_path("谱面")
    if test_chart.exists():
        try:
            test_chart.load()
        except:
            print(i + 1)